In [2]:
import polars as pl 
import pandas as pd
import os
from pathlib import Path

from pydantic import BaseModel, Field
from datetime import date, datetime, time
import pyarrow

In [3]:
rootPath = Path('./../').resolve()
basePath = rootPath / 'src/enbridgescrape'
configsPath = basePath / 'configs'
downloadsPath = rootPath / 'downloads'

In [4]:
df = pl.scan_csv(downloadsPath / 'OA/*.csv')\
        .filter(pl.col('Post_Date').is_not_null())\
        .with_columns(
            pl.col("Post_Date").str.to_date(format="%m-%d-%Y").alias("Post_Date"),
            pl.col("Eff_Gas_Day").str.to_date(format="%m-%d-%Y").alias("Eff_Gas_Day"),
            pl.col("Post_Time").str.to_time(format="%H:%M").alias("Post_Time"),
            pl.col("Eff_Time").str.to_time(format="%H:%M").alias("Eff_Time")).collect()

df = df.with_columns(
            pl.Series(name="Total_Design_Capacity",
                    values=map(lambda x: int(x.replace(',','')) if x.find(',')!=-1 else int(x) ,df['Total_Design_Capacity'])),
            pl.Series(name="Operating_Capacity",
                    values=map(lambda x: int(x.replace(',','')) if x.find(',')!=-1 else int(x) ,df['Operating_Capacity'])),
            pl.Series(name="Total_Scheduled_Quantity",
                    values=map(lambda x: int(x.replace(',','')) if x.find(',')!=-1 else int(x) ,df['Total_Scheduled_Quantity'])),
            pl.Series(name="Operationally_Available_Capacity",
                    values=map(lambda x: int(x.replace(',','')) if x.find(',')!=-1 else int(x) ,df['Operationally_Available_Capacity']))        
    )